In [1]:
using LinearAlgebra
using JSON
using CSVFiles, DataFrames

In [7]:
#Helper Function with Finding Index
function findInd(websites, websiteName)
    for i in 1:size(websites, 1)
        if websites[i] == websiteName
            return i
        end
    end
end

#Generate Markov Matrix Function for fixed example
function generateMarkovMatFixed(websites, websiteLinks)
    numWebsites = size(websites, 1)
    m = zeros((numWebsites, numWebsites))
    for i in 1:numWebsites
        linkedSites = websiteLinks[i]
        for j in 1:size(linkedSites, 1)
            m[CartesianIndex.(findInd(websites, linkedSites[j]), i)] = 1/size(linkedSites, 1)
        end
    end
    return m
end

generateMarkovMatFixed (generic function with 1 method)

In [23]:
#returns array of results where priority is in increasing order (1 is highest priority)
function result(websites, markov)
    pi = markov[:, 1]
    results = zeros((size(websites, 1), 1))
    sort = sortperm(pi)
    for i in 1:size(websites, 1)
        results[sort[i]] = size(websites, 1) - i + 1
    end
    return results
end

result (generic function with 1 method)

In [33]:
#page rank w/damping
function pageRankWithDamping(matrix)
    p = 0.15
    B = ones(size(matrix)) * 1/size(matrix)[1]
    return ((1 - p) * matrix + p * B)^100
end

pageRankWithDamping (generic function with 1 method)

## Connected Graph

In [5]:
#Instantiate input data

websites = ["google.com", "diderot.com", "discord.com", "youtube.com", "gmail.com", "zoom.com"]

websiteLinks= Dict(1=>["diderot.com", "youtube.com", "gmail.com", "zoom.com"],  #google.com
                   2=>["google.com", "gmail.com", "zoom.com"],  #diderot.com
                   3=>["google.com", "youtube.com", "diderot.com"],  #discord.com
                   4=>["gmail.com", "google.com"], #youtube.com
                   5=>["google.com", "diderot.com", "zoom.com", "discord.com"], #gmail.com
                   6=>["youtube.com", "google.com", "gmail.com"]) #zoom.com 

# dicts may be looped through using the keys function:
for k in sort(collect(keys(websiteLinks)))
    print(websites[k], "'s links: ", websiteLinks[k], "\n")
end
println()

google.com's links: ["diderot.com", "youtube.com", "gmail.com", "zoom.com"]
diderot.com's links: ["google.com", "gmail.com", "zoom.com"]
discord.com's links: ["google.com", "youtube.com", "diderot.com"]
youtube.com's links: ["gmail.com", "google.com"]
gmail.com's links: ["google.com", "diderot.com", "zoom.com", "discord.com"]
zoom.com's links: ["youtube.com", "google.com", "gmail.com"]



In [8]:
X = generateMarkovMatFixed(websites, websiteLinks)

6×6 Array{Float64,2}:
 0.0   0.333333  0.333333  0.5  0.25  0.333333
 0.25  0.0       0.333333  0.0  0.25  0.0
 0.0   0.0       0.0       0.0  0.25  0.0
 0.25  0.0       0.333333  0.0  0.0   0.333333
 0.25  0.333333  0.0       0.5  0.0   0.333333
 0.25  0.333333  0.0       0.0  0.25  0.0

In [35]:
M = pageRankWithDamping(X)

6×6 Array{Float64,2}:
 0.243715   0.243715   0.243715   0.243715   0.243715   0.243715
 0.145674   0.145674   0.145674   0.145674   0.145674   0.145674
 0.0731568  0.0731568  0.0731568  0.0731568  0.0731568  0.0731568
 0.144613   0.144613   0.144613   0.144613   0.144613   0.144613
 0.22662    0.22662    0.22662    0.22662    0.22662    0.22662
 0.166221   0.166221   0.166221   0.166221   0.166221   0.166221

In [39]:
#Confirm the columns of M are eigenvectors with eigenvalue 1 of S
#S is the P_{\beta} pos. markov matrix without taking it to a high power

B = ones(size(X)) * 1/size(X)[1]
S = ((1 - 0.15) * X + 0.15 * B)

isapprox(S * M[:, 1], M[:, 1])

true

In [25]:
result(websites, M)

6×1 Array{Float64,2}:
 1.0
 4.0
 6.0
 5.0
 2.0
 3.0

## Disconnected Graph

In [40]:
websitesDisjoint = ["google.com", "diderot.com", "discord.com", "youtube.com", "gmail.com", "zoom.com"]

websiteLinksDisjoint= Dict(1=>["youtube.com", "gmail.com", "zoom.com"],  #google.com
                   2=>["discord.com"],  #diderot.com
                   3=>["diderot.com"],  #discord.com
                   4=>["gmail.com", "google.com"], #youtube.com
                   5=>["google.com", "zoom.com"], #gmail.com
                   6=>["youtube.com", "gmail.com"]) #zoom.com 

# dicts may be looped through using the keys function:
for k in sort(collect(keys(websiteLinksDisjoint)))
    print(websitesDisjoint[k], "'s links: ", websiteLinksDisjoint[k], "\n")
end
println()

google.com's links: ["youtube.com", "gmail.com", "zoom.com"]
diderot.com's links: ["discord.com"]
discord.com's links: ["diderot.com"]
youtube.com's links: ["gmail.com", "google.com"]
gmail.com's links: ["google.com", "zoom.com"]
zoom.com's links: ["youtube.com", "gmail.com"]



In [41]:
Xprime = generateMarkovMatFixed(websitesDisjoint, websiteLinksDisjoint)

6×6 Array{Float64,2}:
 0.0       0.0  0.0  0.5  0.5  0.0
 0.0       0.0  1.0  0.0  0.0  0.0
 0.0       1.0  0.0  0.0  0.0  0.0
 0.333333  0.0  0.0  0.0  0.0  0.5
 0.333333  0.0  0.0  0.5  0.0  0.5
 0.333333  0.0  0.0  0.0  0.5  0.0

In [42]:
Mprime = pageRankWithDamping(Xprime)

6×6 Array{Float64,2}:
 0.169318  0.169318  0.169318  0.169318  0.169318  0.169318
 0.166667  0.166667  0.166667  0.166667  0.166667  0.166667
 0.166667  0.166667  0.166667  0.166667  0.166667  0.166667
 0.140029  0.140029  0.140029  0.140029  0.140029  0.140029
 0.199542  0.199542  0.199542  0.199542  0.199542  0.199542
 0.157778  0.157778  0.157778  0.157778  0.157778  0.157778

In [46]:
#Confirm the columns of Mprime are eigenvectors with eigenvalue 1 of Sprime
#Sprime is the P_{\beta} pos. markov matrix without taking it to a high power

Bprime = ones(size(Xprime)) * 1/size(Xprime)[1]
Sprime = ((1 - 0.15) * Xprime + 0.15 * Bprime)

isapprox(Sprime * Mprime[:, 1], Mprime[:, 1], atol=1e-5)

true

In [46]:
result(websitesDisjoint, Mprime)

6×1 Array{Float64,2}:
 2.0
 4.0
 3.0
 6.0
 1.0
 5.0

## Real Dataset Example

In [24]:
#musae_squirrel_edges.csv
#dataset of page-page networks on specific topics (this dataset is on squirrels)
#Has 5201 nodes (or webpages) and 198,493 edges (undirected links between pages)
#pages indexed from 0, so max page index is 5200

DataFrame(load("musae_squirrel_edges.csv"))

,id1,id2
,Int64,Int64
1,3475,2849
2,3475,3106
3,3475,808
4,3475,4555
5,3475,3563
6,3475,1527
7,3475,3327
8,402,4066
9,402,3908


In [7]:
function countNonZero(arr)
    count = 0
    for elem in arr
        if(elem == 1)
            count = count + 1
        end
    end
    return count
end
        
function CSVGenerateMarkov(text, dim)
    df = DataFrame(load(text))
    m = zeros((dim, dim))
    for row in eachrow(df)
        m[CartesianIndex.(row.id2 + 1, row.id1 + 1)] = 1
        m[CartesianIndex.(row.id1 + 1, row.id2 + 1)] = 1
    end
    for i in 1:(dim)
        nums = countNonZero(m[:, i])
        if(nums != 0)
            m[:, i] = m[:, i] / nums
        end
    end
    return m                 
end

CSVGenerateMarkov (generic function with 1 method)

In [8]:
Xsquirrel = CSVGenerateMarkov("musae_squirrel_edges.csv", 5201)

5201×5201 Array{Float64,2}:
 0.0         0.0  0.0        0.0         0.0  …  0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0     0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0     0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0     0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0     0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0  …  0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0     0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0     0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0     0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0     0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0  …  0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0     0.0       0.0        0.0
 0.0         0.0  0.0        0.0         0.0     0.0       0.0        0.0
 ⋮        

In [9]:
function efficientPageRankWithDamping(matrix, power = 50)
    if(power == 0)
        return ones((size(matrix)[1], 1)) * 1/size(matrix)[1]
    end
    p = 0.15
    B = ones(size(matrix)) * 1/size(matrix)[1]
    starting = ones((size(matrix)[1], 1)) * 1/size(matrix)[1]
    return (1 - p) * matrix * efficientPageRankWithDamping(matrix, power - 1) + p * starting
end
function newPageRankWithDamping(matrix)
    p = 0.15
    B = ones(size(matrix)) * 1/size(matrix)[1]
    starting = ones((size(matrix)[1], 1)) * 1/size(matrix)[1]
    return ((1 - p) * matrix + p * B)^50 * starting
end

newPageRankWithDamping (generic function with 1 method)

In [10]:
@time fastM = efficientPageRankWithDamping(Xsquirrel)

 14.003652 seconds (3.61 M allocations: 40.489 GiB, 21.74% gc time)


5201×1 Array{Float64,2}:
 0.00024764341635520143
 5.883654437119404e-5
 4.7634285996364285e-5
 0.0002175872621968898
 6.446739511168494e-5
 5.502522689030297e-5
 5.29844303208086e-5
 0.00018087817983672888
 4.438339815942886e-5
 6.903322853840814e-5
 0.000178148819967327
 6.476298196547737e-5
 9.122145487608214e-5
 ⋮
 0.0016080909904889744
 0.0002211667874591535
 0.00013855744053936005
 5.70002440104511e-5
 0.0015538421934595196
 0.002328658664354606
 6.231179981845357e-5
 0.0016028765469088397
 0.00031035954421267304
 8.840909946418807e-5
 0.00013517026116400738
 0.00014034259080621187

In [11]:
@time slowM = newPageRankWithDamping(Xsquirrel)

 39.651637 seconds (1.00 M allocations: 2.669 GiB, 0.31% gc time)


5201×1 Array{Float64,2}:
 0.00024764341635520143
 5.8836544371193824e-5
 4.763428599636419e-5
 0.00021758726219688987
 6.446739511168476e-5
 5.502522689030285e-5
 5.298443032080849e-5
 0.00018087817983672896
 4.438339815942876e-5
 6.903322853840803e-5
 0.00017814881996732707
 6.476298196547715e-5
 9.122145487608195e-5
 ⋮
 0.0016080909904889742
 0.0002211667874591536
 0.0001385574405393595
 5.7000244010450954e-5
 0.0015538421934595192
 0.002328658664354599
 6.231179981845338e-5
 0.0016028765469088399
 0.00031035954421267315
 8.840909946418789e-5
 0.00013517026116400703
 0.00014034259080621133

In [12]:
#both stationary distribution vectors are the same
isapprox(slowM, fastM)

true

In [47]:
findmax(fastM)

(0.0051744252297644235, CartesianIndex(4347, 1))

In [15]:
function findMostIncoming(text, dim)
    indCount = zeros((dim + 1, 1))
    df = DataFrame(load(text))
    for row in eachrow(df)
        indCount[row.id1 + 1] += 1
        indCount[row.id2 + 1] += 1
    end
    return indCount  
end

findMostIncoming (generic function with 1 method)

In [16]:
incomeNum = findMostIncoming("musae_squirrel_edges.csv", 5201)

5202×1 Array{Float64,2}:
  154.0
    6.0
   14.0
  150.0
    5.0
   15.0
   12.0
  117.0
    1.0
   21.0
  115.0
    3.0
   31.0
    ⋮
  154.0
   21.0
   17.0
 1120.0
  198.0
   10.0
 1171.0
  240.0
   29.0
   45.0
    7.0
    0.0

In [18]:
findmax(incomeNum)

(2087.0, CartesianIndex(4347, 1))

## Additional Dataset to show difference in efficiency

In [42]:
#same as squirrel dataset, except has 11,631 nodes and 170,918 edges
Xcrocodile = CSVGenerateMarkov("musae_crocodile_edges.csv", 11631)

11631×11631 Array{Float64,2}:
 0.0  0.0  0.0  0.0        0.0  0.0  …  0.0  0.0          0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0     0.0  0.0          0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0     0.0  0.0          0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0     0.0  0.000636943  0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0     0.0  0.0          0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0  …  0.0  0.0          0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0     0.0  0.000636943  0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0     0.0  0.000636943  0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0     0.0  0.0          0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0     0.0  0.0          0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0  …  0.0  0.0          0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0     0.0  0.0          0.0        0.0
 0.0  0.0  0.0  0.0        0.0  0.0     0.0  0.0          0.0        0.0
 ⋮                   

In [63]:
@time fastMCroc = efficientPageRankWithDamping(Xcrocodile)

 62.076384 seconds (1.01 k allocations: 201.609 GiB, 24.20% gc time)


11631×1 Array{Float64,2}:
 8.458833903287631e-5
 1.6652475832516783e-5
 0.00019145935419371507
 6.309512935357827e-5
 2.6358753809455273e-5
 1.7869280451223095e-5
 6.309512935357827e-5
 6.309512935357827e-5
 4.3585351474861785e-5
 1.868684885796903e-5
 0.00010031813437503532
 6.621021877187009e-5
 0.0001206039472007654
 ⋮
 0.0001108899656372685
 8.733521063072639e-5
 7.951975571943295e-5
 5.190180604517787e-5
 0.0008983087876972979
 0.0006439430196361107
 0.00013716483086460707
 4.8871367833648306e-5
 0.0005622327402725814
 0.0029585873984982487
 0.0005973759992581139
 0.00016542783140467032

In [50]:
@time slowMCroc = newPageRankWithDamping(Xcrocodile)

443.405510 seconds (34 allocations: 13.103 GiB, 0.15% gc time)


11631×1 Array{Float64,2}:
 8.458833903287514e-5
 1.665247583251655e-5
 0.0001914593541937125
 6.309512935357727e-5
 2.63587538094549e-5
 1.7869280451222857e-5
 6.309512935357727e-5
 6.309512935357727e-5
 4.358535147486121e-5
 1.868684885796878e-5
 0.00010031813437503455
 6.621021877186913e-5
 0.00012060394720076365
 ⋮
 0.00011088996563726703
 8.733521063072539e-5
 7.951975571943181e-5
 5.190180604517715e-5
 0.0008983087876972857
 0.0006439430196361013
 0.0001371648308646053
 4.8871367833647635e-5
 0.0005622327402725732
 0.002958587398498203
 0.0005973759992581066
 0.00016542783140466845